# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_03_fe_v01_undersampled_10_altbinaria.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
# excluyo al periodo de testing

data = data[data['foto_mes'] != 202107]
data.shape

(363570, 1010)

In [4]:
data['foto_mes'].unique()

array([201906, 201907, 201908, 201909, 201911, 201912, 202001, 202002,
       202003, 202004, 202005, 202007, 202008, 202009, 202010, 202011,
       202012, 202101, 202102, 202103, 202104, 202105, 202106])

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [8]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05),
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria,
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50)
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [9]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia3_lgbm_v01" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-26 19:32:40,476] A new study created in RDB with name: competencia3_lgbm_v01


In [10]:
study.optimize(objective, n_trials=100)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[899]	cv_agg's valid gan_eval: 6.59126e+08 + 2.54889e+06


[I 2024-11-26 19:45:55,998] Trial 0 finished with value: 3295628000.0 and parameters: {'num_leaves': 116, 'learning_rate': 0.009932512892530613, 'min_data_in_leaf': 1546, 'feature_fraction': 0.7381165305260118, 'bagging_fraction': 0.8833974891615759}. Best is trial 0 with value: 3295628000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[521]	cv_agg's valid gan_eval: 6.57611e+08 + 2.7385e+06


[I 2024-11-26 19:50:48,058] Trial 1 finished with value: 3288054000.0 and parameters: {'num_leaves': 56, 'learning_rate': 0.025515934883939627, 'min_data_in_leaf': 1644, 'feature_fraction': 0.7260855580806244, 'bagging_fraction': 0.7525666957111677}. Best is trial 0 with value: 3295628000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[480]	cv_agg's valid gan_eval: 6.55427e+08 + 2.17161e+06


...

[I 2024-11-27 04:53:28,451] Trial 97 finished with value: 3290476000.0 and parameters: {'num_leaves': 95, 'learning_rate': 0.013678335750712057, 'min_data_in_leaf': 1110, 'feature_fraction': 0.6259101957669015, 'bagging_fraction': 0.7403303153853572}. Best is trial 67 with value: 3300451000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[319]	cv_agg's valid gan_eval: 6.58759e+08 + 3.59895e+06


[I 2024-11-27 04:57:30,364] Trial 98 finished with value: 3293794000.0 and parameters: {'num_leaves': 113, 'learning_rate': 0.03155433575023358, 'min_data_in_leaf': 1323, 'feature_fraction': 0.5098834794351872, 'bagging_fraction': 0.7070474282045488}. Best is trial 67 with value: 3300451000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[625]	cv_agg's valid gan_eval: 6.59585e+08 + 2.88812e+06


[I 2024-11-27 05:04:49,269] Trial 99 finished with value: 3297924000.0 and parameters: {'num_leaves': 124, 'learning_rate': 0.014997777390636584, 'min_data_in_leaf': 1411, 'feature_fraction': 0.5320636834265807, 'bagging_fraction': 0.8067046541763531}. Best is trial 67 with value: 3300451000.0.


Analizamos los resultados as usual

In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [13]:
plot_slice(study)

In [14]:
plot_contour(study)

In [15]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [16]:
study.best_trial.params

{'num_leaves': 141,
 'learning_rate': 0.014480948121771996,
 'min_data_in_leaf': 1024,
 'feature_fraction': 0.567467228858195,
 'bagging_fraction': 0.7717340757073016}

In [17]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

618